In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
from state import get_state, queue
import os
import traci

In [41]:
from dotenv import load_dotenv
load_dotenv()


sumo_bin = os.getenv("SUMO")
sumo_gui_bin = os.getenv("SUMO-GUI")
simulConfig = os.getenv("SIMUL-CONFIG2")

print(simulConfig)

nets/LucasAlegre sumo-rl main sumo_rl-nets/Nguyen/nguyen.sumocfg


In [20]:
import numpy as np

from tensorflow.keras import Sequential, layers
from tensorflow.random import set_seed
from collections import deque
from tensorflow import keras
from tensorflow import reduce_sum, reduce_mean, one_hot, GradientTape
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt
import tensorflow as tf

In [21]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [48]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action = Sequential([
    layers.Dense(32,activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_shape=input_shape),
    layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    layers.Dense(n_outputs, activation= 'linear')])

In [22]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:
        Q_values = model_action.predict(state[np.newaxis], verbose=0)[0]
        print(f"state : {state}")
        print(f"Q_values : {Q_values}")
        return Q_values.argmax()  # optimal action according to the DQN

In [23]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [24]:
batch_size = 32
discount_factor = 0.5
optimizer = keras.optimizers.Adam(learning_rate=0.05)
loss_fn = MeanSquaredError()

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states = experiences  # a changer
    next_Q_values = model_action.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = one_hot(actions, n_outputs)
    with GradientTape() as tape:
        all_Q_values = model_action(states)
        Q_values = reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model_action.trainable_variables)
    optimizer.apply_gradients(zip(grads, model_action.trainable_variables))

In [ ]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
replay_buffer = deque(maxlen=10000)
epoch = 50

for episode in range(epoch):
    #SEED
    seed_value = np.random.randint(1000)
    #seed_value = 41
    sumoCmd = [sumo_bin, "-c", simulConfig2x2, "--start", "--seed", str(seed_value), '--no-warnings']

    if traci.isLoaded():
        traci.close()
    traci.start(sumoCmd)
    lane_ids =  traci.lane.getIDList()
    # print(lane_ids[0])

    trafic_light_ids = traci.trafficlight.getIDList()

    # state = np.array(queue(lane_ids))
    state=np.array(get_state(lane_ids))
    action=-1
    # print(state)
    wait_times.append(0)
    for step in range(80000): ## TO CHANGED
        epsilon = max(1 - episode / epoch, 0.01)

        if step%2000 == 0:
            #print(f"longeur du buffer :{len(replay_buffer)}")
            ########################################################################
            ##Calcul de la reward
            # next_state = np.array(queue(lane_ids))
            next_state = np.array(get_state(lane_ids))
            # reward = calculate_reward(values, reward, total_reward)[0]
            # reward=(np.sum(state)-np.sum(next_state))
            reward = np.sum(state[:24])- np.sum(next_state[:24])
            replay_buffer.append((state, action, reward, next_state))

            list_values.append(queue(lane_ids))
            #########################################################################
            state=next_state
            action = epsilon_greedy_policy(state, epsilon)
            #print("action", action)
            traci.trafficlight.setPhase(trafic_light_ids[0],2*action)

            if len(replay_buffer) >= batch_size*10:
                training_step(batch_size)
                # new_weights = model_action.get_weights()
                # weights.append(new_weights)
                #print(f"Episode {episode}: new weights = {new_weights}")
            # else:
            #     print(f"Episode {episode}: pas assez de données dans le replay buffer.")
            # print(values)
            # if list_values:
            #    # print(f'list values {list_values[-1]}')


        traci.simulationStep()
    print(f'episode : {episode}')
    traci.close()


 Retrying in 1 seconds
Step #720.00 (2ms ~= 10.00*RT, ~577000.00UPS, TraCI: 0ms, vehicles TOT 1839 ACT 577 BUF 46 #180.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 485 ACT 254 BUF 178)               0.00 (1ms ~= 10.00*RT, ~379000.00UPS, TraCI: 7ms, vehicles TOT 970 ACT 379 BUF 389 (1ms ~= 10.00*RT, ~490000.00UPS, TraCI: 7ms, vehicles TOT 1434 ACT 490 BUF 62episode : 0
Step #800.00 (2ms ~= 5.00*RT, ~306000.00UPS, TraCI: 0ms, vehicles TOT 2011 ACT 612 BUF 289
 Retrying in 1 seconds


state : [ 0 18  4  0 24  0  0 17 10  0 20  0  0  0  0  0  0  0  0  0  0  0  0  0
  7 25 13  9 31 10  7 24 15  9 28 11  3  2 13  4  0  0  4  2 13  8  0  0]
Q_values : [-6.094483   5.292813  -3.7523992  4.2365594]
Step #720.00 (1ms ~= 5.00*RT, ~345500.00UPS, TraCI: 0ms, vehicles TOT 1844 ACT 691 BUF 558episode : 1
Step #800.00 (1ms ~= 10.00*RT, ~694000.00UPS, TraCI: 0ms, vehicles TOT 1951 ACT 694 BUF 45
 Retrying in 1 seconds


state : [ 0  0 49  0 26 19  0  0 57  0 26 23  0  0  0  0  0  0  0  0  0  0  0  0
 10  5 55 10 45 24 11 10 60  6 59 30  7  6  4  2  1  0  6  6  7  2  0  0]
Q_values : [-13.599979   14.87446    -0.9381483   8.9843445]
state : [ 0 43 43  0 21  0  0 41 38 20 39  6  0  0  0  0  0  0  0  0  0  0  0  0
  7 51 65  8 66 53  6 47 74 37 69 55  4  1  9  2  2  1  0  0  9  5  4  0]
Q_values : [-6.304126  12.150341   7.3553023  9.31376  ]
Step #720.00 (1ms ~= 10.00*RT, ~661000.00UPS, TraCI: 0ms, vehicles TOT 1852 ACT 661 BUF 51episode : 2
Step #800.00 (1ms ~= 10.00*RT, ~655000.00UPS, TraCI: 0ms, vehicles TOT 1976 ACT 655 BUF 38
 Retrying in 1 seconds


state : [ 1 30 58  0 17 22  6 28 62  3 29 34  0  0  0  0  0  0  0  0  0  0  0  0
 10 70 66 10 51 55 13 55 67 21 57 71  4  4  3  6  4  5  0  0  5  6 10  2]
Q_values : [-6.3767114  6.0758324 10.571052   6.893047 ]
Step #720.00 (1ms ~= 10.00*RT, ~640000.00UPS, TraCI: 0ms, vehicles TOT 1904 ACT 640 BUF 49episode : 3
Step #800.00 (1ms ~= 10.00*RT, ~644000.00UPS, TraCI: 1ms, vehicles TOT 2013 ACT 644 BUF 38
 Retrying in 1 seconds


state : [57 79  0  0 18 27  1 82  0  0 24 27  0  0  0  0  0  0  0  0  0  0  0  0
 65 79 50  6 28 42 16 82 52 12 31 31  1  0  0  3  1 14  3  0  0  0  0 12]
Q_values : [ 2.3924289  2.5368636 -9.938291  14.59186  ]
state : [25 61 17  0 57 28 23 38 15  0 48 22  0  0  0  0  0  0  0  0  0  0  0  0
 38 70 41  7 63 50 42 68 58  9 50 43  3  0  9  0  1  2  5  0  8  6  2  6]
Q_values : [-1.9955777  7.751885  -7.888639   9.788451 ]
Step #720.00 (1ms ~= 10.00*RT, ~593000.00UPS, TraCI: 1ms, vehicles TOT 1758 ACT 593 BUF 60episode : 4
Step #800.00 (1ms ~= 10.00*RT, ~631000.00UPS, TraCI: 1ms, vehicles TOT 1914 ACT 631 BUF 44
 Retrying in 1 seconds


...

state : [ 0  3  4  0  5  0  0  3  1  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0
  9 12 12 10 12  4  6  7  8 13 10  7  4  0  5  6  0  0  4  1  6  6  0  0]
Q_values : [-15.46629  -19.280258 -33.533607 -16.258364]
state : [ 0  0  9  0 11  1  0  0  7  0  8  6  0  0  0  0  0  0  0  0  0  0  0  0
  5  5 18  7 19  9  8  8  9  5 13 12  7  4  1  7  0  0  7  7  6  3  2  1]
Q_values : [-15.370116 -18.044842 -34.81088  -16.94133 ]
state : [ 0  5 33  0  6 21  0  3 22  0  1 19  0  0  0  0  0  0  0  0  0  0  0  0
  6 15 39 10 27 24  8 12 30  9 24 23  2  2  0  0  7  0  2  0  0  4  9  2]
Q_values : [-13.845331 -22.23216  -35.51748  -12.499195]
state : [ 0 12 38  0 10  0  0  5 29  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0
  8 21 45 10 32 18  9 15 36 10 31 17  4  1  6  3  5  0  4  1  8  5  1  1]
Q_values : [-18.237549 -24.874493 -36.17732  -16.288565]
state : [ 0 14 66  0 52  0  0 21 53  0 47  0  0  0  0  0  0  0  0  0  0  0  0  0
  5 30 68  5 60  9  9 25 55 11 50  5  1  1 10  2  0  0  4  0 12  7  0  

state : [ 0  7  9  0  2  0  0  6  8  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0
  5 18 13  5 11  8  6 12 14 10 10  2  0  0  3  1  0  0  5  0 10  2  1  0]
Q_values : [-7.8571844  4.9677114 -9.579395  -4.2907796]
state : [ 0 14  0  0  9  7  0  9  0  0  7  2  0  0  0  0  0  0  0  0  0  0  0  0
  8 19 10  9 14 12  6 15  8  9 13  6  2  0  0  4  1  9  4  2  3  2  0  7]
Q_values : [-8.631732  -2.3758366 -8.963022  -5.009456 ]
state : [ 0 17  0  0 14 11  0 14  0  0 12  6  0  0  0  0  0  0  0  0  0  0  0  0
  9 26  5 11 21 14 10 18  2 13 16  9  3  0  0  2  0 10  4  0  0  2  0 12]
Q_values : [-3.540867  -5.4296017 -1.9173214 -6.1127534]
state : [ 0 26  8  0 33 10  0 24  4  1 40  5  0  0  0  0  0  0  0  0  0  0  0  0
  7 46 27  7 40 31  7 31 25  7 46 12  1  0  2  7  1  8  5  0  5  2  0  8]
Q_values : [  4.4863343 -11.358449   -3.6332824 -11.646008 ]
state : [ 0 42  2  0 10 45  0 33  0  0 14 25  0  0  0  0  0  0  0  0  0  0  0  0
  6 49 31 12 43 48 12 37 24  9 52 33  1  1  0  4  2  9  1  0  0  1 

state : [ 0  6  0  0  1  6  0  7  0  0  2  6  0  0  0  0  0  0  0  0  0  0  0  0
  5 16 10  8  8 12  7 13  8  8  9  8  4  0  0  1  0  7  2  0  0  4  0  5]
Q_values : [-17.127747  -8.687773 -12.188342  -9.817127]
state : [ 0  0  7  0 15 20  0  0  3  0 13 17  0  0  0  0  0  0  0  0  0  0  0  0
  8 13 18 10 20 25  5 19 12 11 20 19  3  7  1  3  2  1  6  7  3  1  0  1]
Q_values : [-5.040358  -9.325022  -3.8957658 -9.023049 ]
state : [ 0 14  0  0 11 29  0 21  0  0 10 24  0  0  0  0  0  0  0  0  0  0  0  0
 11 18 18  8 19 33  6 29 12 10 19 28  4  0  0  3  2 10  7  0  0  4  4  8]
Q_values : [-3.3179395 -8.061001  -3.0768063 -8.163879 ]
state : [ 0 18 10  0  0 33  0 27 10  0  0 27  0  0  0  0  0  0  0  0  0  0  0  0
  7 22 23  6 17 36  8 33 19  8 14 33  6  0  0  3  8  5  6  0  0  4 10  6]
Q_values : [ -6.345225    5.0058393 -16.504454   -1.6518462]
state : [ 0 21 12  0 10 27  0 37 11  0  0 27  0  0  0  0  0  0  0  0  0  0  0  0
  4 32 21  7 34 53  9 43 17  9 28 40  2  0  1  5  5 10  3  0  0  6 

state : [ 0  6  5  0  4  0  0  5  7  0  5  0  0  0  0  0  0  0  0  0  0  0  0  0
 12 15 10  7 13  5 10 12 11 10 12  2  2  0  8  3  1  0  2  0  5  2  0  0]
Q_values : [ -2.1567123 -12.034686  -12.349913   -7.29031  ]
state : [ 0 15 11  0  0 22  0  5 11  0  0 23  0  0  0  0  0  0  0  0  0  0  0  0
  5 19 26  8 11 27  9 12 23  8 17 29  4  0  0  3  9  6  3  1  0  2  7  6]
Q_values : [ -9.375426  -10.380629  -18.515915  -11.1139765]
state : [ 0  0 32  0 13 27  0  0 25  0 17 34  0  0  0  0  0  0  0  0  0  0  0  0
 13  3 33  7 20 34  6  6 34  7 23 40  4  6  3  5  1  1  6 10  6  3  0  0]
Q_values : [-12.722879   -3.7636836 -14.917477  -11.375363 ]
Interrupt signal received, trying to exit gracefully.0ms, vehicles TOT 832 ACT 341 BUF 448


KeyboardInterrupt: 

In [36]:
rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
# replay_buffer = deque(maxlen=2000)

#SEED
seed_value = np.random.randint(1000)
#seed_value = 41
sumoCmd = [sumo_gui_bin, "-c", simulConfig, "--start", "--seed", str(seed_value)]

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
# print(lane_ids[0])
# for lane in lane_ids:
#     print(traci.lane.getLastStepVehicleNumber(lane))
# north_lane = traci.lane.getLastStepVehicleNumber("N_0")
# south_lane = traci.lane.getLastStepVehicleNumber("S_0")
# east_lane = traci.lane.getLastStepVehicleNumber("E_0")
# west_lane = traci.lane.getLastStepVehicleNumber("W_0")
trafic_light_ids = traci.trafficlight.getIDList()

state = np.array(get_state(lane_ids))
action=-1
# print(state)
wait_times.append(0)
for step in range(100000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)
    nom_du_feu= traci.trafficlight.getIDList()[0]

    if step%2000 == 0:
        state=np.array(get_state(lane_ids))
        #action = epsilon_greedy_policy(state,0)*2
        action = np.random.randint(8)
        print("action", action)
        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu))

        #print(traci.trafficlight.getAllProgramLogics(nom_du_feu)[0].phases[action])
        traci.trafficlight.setPhase(trafic_light_ids[0],action)
    traci.simulationStep()

traci.close()


 Retrying in 1 seconds
action 6
action 3
action 1
action 5
action 4
action 7
action 1
action 1
action 6
action 4
action 7
action 2
action 6


X Fatal error.


FatalTraCIError: Connection closed by SUMO.